# Metadynamics Exercise


In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import glob
import subprocess

In [ ]:
%%bash
source 

In [ ]:
plt.rcParams['figure.dpi'] = 500

In [ ]:
home_dir=os.getcwd()
print(home_dir)

## System

In this tutorial, we will consider the association/dissociation of NaCl in aqueous solution. The system consists of 1 Na atom, 1 Cl atom, and 107 water molecules for a total of 323 atoms. In an effort to speed up the simulations, we employ a rather small water box, and thus need to employ smaller cutoffs than usually used. Therefore, this simulation setup should not be used in production runs. 

We will run the simulations at 300 K in the NPT ensemble. 

### Collective Variables (CVs)

#### Na-Cl Distance

The most relevant CV for this system is the distance between the Na and Cl atoms. If we look at the `NaCl_StartingStructure-1.gro` file, we can see Na atom is number 322 while the Cl atom is number 323. Thus, we can define the distance CV in PLUMED as

`dist: DISTANCE ATOMS=322,323`

#### Na Solvation 

Furthermore, the NaCl association/dissociation is coupled to the collective motion of the solvent. To measure that, we will use a CV that measures the solvation of the Na atom. For this, we employ the coordination number of the Na atom with respect to the oxygens of the water molecules that we define in PLUMED as

<code>
coord: COORDINATION ...
   GROUPA=322   
   GROUPB=1-321:3 
   SWITCH={RATIONAL R_0=0.315 D_MAX=0.5 NN=12 MM=24}  
   NLIST 
   NL_CUTOFF=0.55 
   NL_STRIDE=10    
...
</code>

Here we are using a rational switching function that measures the coordiation number in a smooth way. See the manual for __[COORDINATION](https://www.plumed.org/doc-v2.8/user-doc/html/_c_o_o_r_d_i_n_a_t_i_o_n.html)__ for further information. 

In [ ]:
# kB*T at 300 K in kJ/mol
kBT=2.494353
beta=1/kBT

# Exercise 1 - Unbiased Simulation

In this exercise we will perform an unbiased simulations of the system and monitor the two CVs defined above.

We start copying the files from `Exercise-X` to new folder `Exercise-1` where we will perform the simulation. 

In [ ]:
%cp -r Exercise-X/ Exercise-1 

In [ ]:
%cd Exercise-1/

In [ ]:
%ls

We need to start by setting the MD parameters for GROMACS that are set in the `MD-NPT.mdp` file. 
We will generate this file below by using `cat` with the `EOF` command. 

We are using a MD timestep of 0.002 ps.

We will run a simulation for 1000 ps, or (1000 ps)/(0.002 ps) = 500000 MD steps. This is set by the `nsteps` parameter. 

In [ ]:
%%bash
cat <<EOF > MD-NPT.mdp
integrator = md
dt = 0.002
nsteps = 500000
cutoff-scheme = Verlet
coulombtype = PME
rlist = 0.6
rcoulomb = 0.6
rvdw = 0.6
constraints = h-bonds
tcoupl =  V-rescale
ref_t = 300
tau-t = 1.0
tc-grps = System
gen-vel = no
gen-temp = 300
gen-seed = 2561
DispCorr = AllEnerPres
pcoupl = Parrinello-Rahman
pcoupltype = isotropic
ref-p = 1.01325
compressibility = 4.5e-5
nstxout-compressed = 250
nstxout = 0
nstvout = 0
EOF

In [ ]:
# to check that the MD-NPT.mdp file was succesfully generated
%ls

We then need to generate a GROMACS tpr file that we then run using `mdrun`. For this we need to have the mdp parameter file `MD-NPT.mdp`, a topology file `NaCl.top`, and a inital geometry `NaCl_StartingStructure-1.gro`. 

The script `generate-tpr-file.sh` will generate the tpr file. 

In [ ]:
%%bash
cat generate-tpr-file.sh

In [ ]:
%%bash
./generate-tpr-file.sh "NaCl_StartingStructure-1.gro"

In [ ]:
# To check that the tpr file was succesfully generated 
%ls

We then need to generate the PLUMED input file. 

In this file, we define the CVs discussed above and print them to a file called `colvar.dat` by using the `PRINT` action. The `STRIDE=250` keyword defines that we print the CV values every 250 MD steps. 

In [ ]:
%%bash
cat <<EOF > plumed.dat
# vim:ft=plumed

# Distance between Na and Cl atoms
dist: DISTANCE ATOMS=322,323

# Solvation of Na atom
COORDINATION ...
  LABEL=coord
  GROUPA=322
  GROUPB=1-321:3
  SWITCH={RATIONAL R_0=0.315 D_MAX=0.5 NN=12 MM=24}
  NLIST
  NL_CUTOFF=0.55
  NL_STRIDE=10
... COORDINATION

PRINT ARG=dist,coord FILE=colvar.data STRIDE=250
EOF

In [ ]:
%ls

We then have all the files need to run the simulation. 

We will use the `run.sh` script to run the simulation. 

This script also 

In [ ]:
%cat run.sh

In [ ]:
%%bash
./run.sh

As we can see, this simulation has generated various files

In [ ]:
%ls

We can look at the colvar file `colvar.data`, where we can see that the first line is a header line that shows what variables are in which column.

In [ ]:
%%bash
head colvar.data

We define two python functions to read the data from the colvar file, and to plot it, that we will use in the following.

In [ ]:
def get_colvardata(filename_colvar):
    with open(filename_colvar,'r') as colvar_file:
        colvar_labels = colvar_file.readline().split()[2:]
    colvar_data=np.loadtxt(filename_colvar)
    return (colvar_labels,colvar_data)

In [ ]:
def plot_colvardata(filename_colvar,column):
    (colvar_labels,colvar_data)=get_colvardata(filename_colvar)
    plt.plot(colvar_data[:,0],colvar_data[:,column-1],".")
    plt.title(colvar_labels[column-1])
    plt.xlabel("Time [ps]")
    plt.ylabel(colvar_labels[column-1])
    plt.show()

Let us plot the time series of the distance, which is column 2 in the colvar file.

In [ ]:
plot_colvardata("colvar.data",2)

We can see that there are some transitions between the associated and dissociated state.  

We want to estimate the fluctations (i.e., standard deviations) of the distance CV to select the sigma value for the metadynamics simulations. However, we need to do that separately for the associated and dissociated state. We use a distance of 0.35 to separate the two states. We can do that with following python code. 

In [ ]:
(labels,colvar_data)=get_colvardata("colvar.data")
# distance is 2nd column, 1 in python indexing 
time = colvar_data[:,0]
dist = colvar_data[:,1]

dist_separate=0.35
dist_assoc=   dist[dist<dist_separate]
dist_dissoc=  dist[dist>dist_separate]
time_assoc=   time[dist<dist_separate]
time_dissoc=  time[dist>dist_separate]

plt.plot(time_assoc,dist_assoc,'.',label="Associated")
plt.plot(time_dissoc,dist_dissoc,'.',label="Dissociated")
plt.xlabel("Time [ps]")
plt.ylabel("Distance [nm]")
plt.legend()
plt.show()

print("Distance CV")
print(" Associated State")
print("   Average: {:.3f}".format(np.average(dist_assoc)))
print("   Standard Deviation: {:.3f}".format(np.std(dist_assoc)))
print(" Dissociated State")
print("   Average: {:.3f}".format(np.average(dist_dissoc)))
print("   Standard Deviation: {:.3f}".format(np.std(dist_dissoc)))


When determining the approriate value for the width of the Gaussian (i.e., SIGMA parameter) for a CV, we need to consider the lower value of the fluctions, which is in this case is for the more ordered associated state. 

Thus, from these results, we can determine that an approriate value for the width of the Gaussian (i.e., SIGMA parameter), for the distance CV is around 0.01 nm. If we would use the value for the dissociated state of around 0.2 nm it is likely that we smear out the free energy surface. 

**Thus we get that an approriate value is around 0.01 nm**

Before continuing, we should look at the behavior of the solvation CV (the 3rd column)

In [ ]:
plot_colvardata("colvar.data",3)

We can also look at the correlation of the distance and solvation CVs

In [ ]:
(labels,colvar_data)=get_colvardata("colvar.data")
plt.plot(colvar_data[:,1],colvar_data[:,2],'.')
plt.xlabel(labels[1])
plt.ylabel(labels[2])
plt.show()

Which are tightly correlated, as expected

In [ ]:
%cd ..

# Exercise 2 - Biasing Distance CV

We will now consider exercise 2 where we will bias the distance CV. 

We will start with copying a new folder for the runs and generating all the GROMACS files. 

Now we will run the simulation for 5000 ps (2500000 MD steps). 

In [ ]:
os.chdir(home_dir)

In [ ]:
%cp -r Exercise-X Exercise-2 

In [ ]:
%cd Exercise-2/

In [ ]:
%%bash
cat <<EOF > MD-NPT.mdp
integrator = md
dt = 0.002
nsteps = 2500000
cutoff-scheme = Verlet
coulombtype = PME
rlist = 0.6
rcoulomb = 0.6
rvdw = 0.6
constraints = h-bonds
tcoupl =  V-rescale
ref_t = 300
tau-t = 1.0
tc-grps = System
gen-vel = no
gen-temp = 300
gen-seed = 2561
DispCorr = AllEnerPres
pcoupl = Parrinello-Rahman
pcoupltype = isotropic
ref-p = 1.01325
compressibility = 4.5e-5
nstxout-compressed = 250
nstxout = 0
nstvout = 0
EOF

In [ ]:
%%bash
./generate-tpr-file.sh "NaCl_StartingStructure-1.gro"

Now we need to setup the PLMUED input file for the Metadynamics runs. For that we add a `METAD` action to the input file. 

Add more on METAD parameters


In [ ]:
%%bash
cat <<EOF > plumed.dat
# vim:ft=plumed

# Distance between Na and Cl atoms
dist: DISTANCE ATOMS=322,323

# Solvation of Na atom
COORDINATION ...
  LABEL=coord
  GROUPA=322
  GROUPB=1-321:3
  SWITCH={RATIONAL R_0=0.315 D_MAX=0.5 NN=12 MM=24}
  NLIST
  NL_CUTOFF=0.55
  NL_STRIDE=10
... COORDINATION

METAD ...
  LABEL=mtd
  ARG=dist
  PACE=500
  SIGMA=0.02
  HEIGHT=1.25
  BIASFACTOR=5
  GRID_MIN=0.0
  GRID_MAX=1.5
  CALC_RCT
  FILE=hills.data
... METAD

PRINT ARG=dist,coord,mtd.* FILE=colvar.data STRIDE=250
EOF

In [ ]:
%ls

In [ ]:
%%bash
./run.sh

In [ ]:
%ls

Let start with looking at the time series of the distance CV that we are biasing

In [ ]:
plot_colvardata("./colvar.data",2)

We can see that now there are much more frequent transtions between the two states 

We now also have the `hills.data` file that includes the added Gaussians. 

In [ ]:
%%bash
head hills.data

We should also look at the height of the added Gaussians, which should decrease as the simulation progresses and go to zero in the long time limit. This is in the 4th column in the `hills.data` file 

In [ ]:
plot_colvardata("./hills.data",4)

We can obtain the FES from the added Gaussian by using the `sum_hills` command line tool of PLUMED 

In [ ]:
%%bash
mkdir fes
plumed sum_hills --hills hills.data --outfile fes/fes.data 
plumed sum_hills --hills hills.data --mintozero --outfile fes/fes.mintozero.data 

In [ ]:
# Plot FES 

fes=np.loadtxt("fes/fes.data")
plt.plot(fes[:,0],fes[:,1]/kBT,'-')
plt.xlabel("Na-Cl Distance [nm]")
plt.ylabel("Free Energy [kBT]")

In [ ]:
%%bash
mkdir fes-stride-500
plumed sum_hills --hills hills.data --mintozero --stride 500 --outfile fes-stride-500/fes.stride-

In [ ]:
fes_stride=500
for i, f in enumerate(glob.glob("fes-stride-500/fes.stride-*")):
    time=(i+1)*fes_stride
    fes=np.loadtxt(f)
    plt.plot(fes[:,0],fes[:,1]/kBT,'-',label="{:4d} ps".format(time))    
plt.xlabel("Na-Cl Distance [nm]")
plt.ylabel("Free Energy [kBT]")
plt.legend()
plt.show()

## Free Energy Difference 

In [ ]:
# Plot PDF

fes=np.loadtxt("fes/fes.data")
pdf=np.exp(-beta*(fes[:,1]-np.min(fes[:,1])))
plt.plot(fes[:,0],pdf,'-')
plt.xlabel("Na-Cl Distance [nm]")
plt.ylabel("Probablity")

plt.ylim([0,1.1])

barrier_location=0.36
plt.axvspan(0.2,barrier_location,alpha=0.2,color='orange')
plt.axvspan(barrier_location,np.max(fes[:,0]),alpha=0.2,color='green')

In [ ]:
def calc_free_energy_difference(fes_file):
    barrier_location=0.36
    fes_data = np.loadtxt(fes_file)
    distance = fes_data[:,0]
    fes = fes_data[:,1]-np.min(fes_data[:,1])
    prob = np.exp(-beta*fes)
    prob_A = 0.0
    prob_B = 0.0
    for i in range(prob.size):
        if(distance[i]<barrier_location): 
            prob_A += prob[i]
        if(distance[i]>barrier_location): 
            prob_B += prob[i]
    free_energy_difference = -np.log(prob_A/prob_B)
    return free_energy_difference

In [ ]:
%%bash
mkdir fes-stride-100
plumed sum_hills --hills hills.data --mintozero --stride 100 --outfile fes-stride-100/fes.stride-

In [ ]:
fe_time=[]
fe_diff=[]
fes_stride=100
for i, f in enumerate(glob.glob("fes-stride-100/fes.stride-*")):
    time=(i+1)*fes_stride
    fe_time.append(time)
    fe_diff.append(calc_free_energy_difference(f))

plt.plot(fe_time,fe_diff)
plt.xlabel("Time [ps]")
plt.ylabel("Free Energy Difference [kBT]")
plt.title("Free Energy Difference")
plt.ylim([-4.0,4.0])



# Exercise 3 - Reweighting

## Reweighthing on distance CV

In [ ]:
%%bash
head colvar.data

In [ ]:
plot_colvardata("colvar.data",4)
plot_colvardata("colvar.data",5)
plot_colvardata("colvar.data",6)

In [ ]:
%%bash
mkdir fes-reweight


In [ ]:
%%bash
cat <<EOF > fes-reweight/plumed_reweight.dat
# vim:ft=plumed

dist:   READ FILE=fes-reweight/colvar_reweight.data IGNORE_TIME VALUES=dist
coord:  READ FILE=fes-reweight/colvar_reweight.data IGNORE_TIME VALUES=coord
mtd:    READ FILE=fes-reweight/colvar_reweight.data IGNORE_TIME VALUES=mtd.rbias

weights: REWEIGHT_BIAS TEMP=300 ARG=mtd.rbias

HISTOGRAM ...
  ARG=dist
  GRID_MIN=0.2
  GRID_MAX=1.5
  GRID_BIN=100
  KERNEL=DISCRETE
  LOGWEIGHTS=weights
  LABEL=hg_dist
... HISTOGRAM

fes_dist: CONVERT_TO_FES GRID=hg_dist TEMP=300 MINTOZERO
DUMPGRID GRID=fes_dist FILE=fes-reweight/fes-reweight.dist.data FMT=%24.16e
EOF

In [ ]:
for i in range(0,1001,100):
    subprocess.run("rm -f fes-reweight/fes-reweight.dist.data".split(),stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
    subprocess.run("rm -f fes-reweight/colvar_reweight.data".split(),stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
    subprocess.run("./trim-colvar-file.py --colvar-file colvar.data --output-file fes-reweight/colvar_reweight.data --time-min {0}".format(i).split(),stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
    subprocess.run("plumed driver --plumed fes-reweight/plumed_reweight.dat --noatoms".split(),stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
    fes_data=np.loadtxt("fes-reweight/fes-reweight.dist.data")
    plt.plot(fes_data[:,0],fes_data[:,1]/kBT,label="Trim {} ps".format(i))
    plt.xlabel("Na-Cl distance [nm]")  
    plt.ylabel("Free Energy [kBT]")
    plt.title("Reweighted FES")
    plt.legend()
    subprocess.run("rm -f fes-reweight/fes-reweight.dist.data".split())
    subprocess.run("rm -f fes-reweight/colvar_reweight.data".split())
    plt.ylim([0,18])

In [ ]:
time_trim=500
subprocess.run("./trim-colvar-file.py --colvar-file colvar.data --output-file fes-reweight/colvar_reweight.data --time-min {0}".format(time_trim).split(),stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
subprocess.run("plumed driver --plumed fes-reweight/plumed_reweight.dat --noatoms".split(),stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)



In [ ]:
fes_reweight=np.loadtxt("fes-reweight/fes-reweight.dist.data")
fes_sumhills=np.loadtxt("fes/fes.mintozero.data")

plt.plot(fes_reweight[:,0],fes_reweight[:,1]/kBT,'-',label="Reweighted")
plt.plot(fes_sumhills[:,0],fes_sumhills[:,1]/kBT,'-',label="From bias potential")
plt.xlabel("Na-Cl Distance [nm]")
plt.ylabel("Free Energy [kBT]")


## Reweighting on both distance and solvation CV

In [ ]:
%%bash
cat << EOF > fes-reweight/plumed_reweight.2D.dat 
# vim:ft=plumed

dist:   READ FILE=fes-reweight/colvar_reweight.data IGNORE_TIME VALUES=dist
coord:  READ FILE=fes-reweight/colvar_reweight.data IGNORE_TIME VALUES=coord
mtd:    READ FILE=fes-reweight/colvar_reweight.data IGNORE_TIME VALUES=mtd.rbias

weights: REWEIGHT_BIAS TEMP=300 ARG=mtd.rbias

HISTOGRAM ...
  ARG=dist,coord
  GRID_MIN=0.2,2.5
  GRID_MAX=0.9,7.5
  GRID_BIN=200,200
  BANDWIDTH=0.004,0.04
  LOGWEIGHTS=weights
  LABEL=hg_dist_coord
... HISTOGRAM

fes_dist_coord: CONVERT_TO_FES GRID=hg_dist_coord TEMP=300 MINTOZERO
DUMPGRID GRID=fes_dist_coord FILE=fes-reweight/fes-reweight.dist-coord.data FMT=%24.16e
EOF

In [ ]:
%%bash
plumed driver --plumed fes-reweight/plumed_reweight.2D.dat --noatoms &> /dev/null

In [ ]:
fes_data = np.loadtxt("./fes-reweight/fes-reweight.dist-coord.data")
distance = fes_data[:,0].reshape(201,201)
coord =    fes_data[:,1].reshape(201,201)
fes_2d =   fes_data[:,2].reshape(201,201)
from matplotlib import cm
plt.contourf(distance,coord,fes_2d/kBT, levels=np.linspace(0,10,101), cmap=cm.jet)
plt.xlabel("Na-Cl distance [nm]")
plt.ylabel("Solvation of Na")
plt.title("FES - Reweighting")
plt.colorbar(label="Free Energy [kBT]", ticks=range(0,11))

# Exercise 4 - Biasing both distance and solvation CV

In [ ]:
os.chdir(home_dir)

In [ ]:
%%bash
cp -r Exercise-X Exercise-4


In [ ]:
%cd Exercise-4/

In [ ]:
%%bash
cat <<EOF > MD-NPT.mdp
integrator = md
dt = 0.002
nsteps = 2500000
cutoff-scheme = Verlet
coulombtype = PME
rlist = 0.6
rcoulomb = 0.6
rvdw = 0.6
constraints = h-bonds
tcoupl =  V-rescale
ref_t = 300
tau-t = 1.0
tc-grps = System
gen-vel = no
gen-temp = 300
gen-seed = 2561
DispCorr = AllEnerPres
pcoupl = Parrinello-Rahman
pcoupltype = isotropic
ref-p = 1.01325
compressibility = 4.5e-5
nstxout-compressed = 250
nstxout = 0
nstvout = 0
EOF

In [ ]:
%%bash
./generate-tpr-file.sh NaCl_StartingStructure-1.gro


In [ ]:
%%bash
cat <<EOF > plumed.dat
# vim:ft=plumed

# Distance between Na and Cl atoms
dist: DISTANCE ATOMS=322,323

# Solvation of Na atom
COORDINATION ...
  LABEL=coord
  GROUPA=322
  GROUPB=1-321:3
  SWITCH={RATIONAL R_0=0.315 D_MAX=0.5 NN=12 MM=24}
  NLIST
  NL_CUTOFF=0.55
  NL_STRIDE=10
... COORDINATION

METAD ...
  LABEL=mtd
  ARG=dist,coord
  PACE=500
  SIGMA=0.02,0.1 
  HEIGHT=1.25
  BIASFACTOR=5
  GRID_MIN=0.0,2.0
  GRID_MAX=1.5,9.0
  CALC_RCT
  FILE=hills.data
... METAD

PRINT ARG=dist,coord,mtd.* FILE=colvar.data STRIDE=250
EOF

In [ ]:
%%bash
./run.sh

In [ ]:
plot_colvardata("./colvar.data",2)
plot_colvardata("./colvar.data",3)
plot_colvardata("./hills.data",6)

In [ ]:
%%bash
rm -rf fes-stride-1000 
mkdir fes-stride-1000
plumed sum_hills --hills hills.data --mintozero --stride 1000 --kt 2.494353 --idw dist --outfile fes-stride-1000/fes.dist.stride-
plumed sum_hills --hills hills.data --mintozero --stride 1000 --kt 2.494353 --idw coord --outfile fes-stride-1000/fes.coord.stride-

In [ ]:
fes_stride=1000
num_fesfile=len(glob.glob("fes-stride-1000/fes.dist.stride-*"))
for i in range(num_fesfile):  
    if i == num_fesfile-1: continue
    fesfile="fes-stride-1000/fes.dist.stride-{}.dat".format(i)
    time=(i+1)*fes_stride
    fes=np.loadtxt(fesfile)
    plt.plot(fes[:,0],fes[:,1]/kBT,'-',label="{:4d} ps".format(time))    
plt.xlabel("Na-Cl Distance [nm]")
plt.ylabel("Free Energy [kBT]")
plt.legend()
plt.show()

In [ ]:
fes_stride=1000
num_fesfile=len(glob.glob("fes-stride-1000/fes.coord.stride-*"))
for i in range(num_fesfile):  
    if i == num_fesfile-1: continue
    fesfile="fes-stride-1000/fes.coord.stride-{}.dat".format(i)
    time=(i+1)*fes_stride
    fes=np.loadtxt(fesfile)
    plt.plot(fes[:,0],fes[:,1]/kBT,'-',label="{:4d} ps".format(time))    
plt.xlabel("Na Solvation")
plt.ylabel("Free Energy [kBT]")
plt.legend()
plt.show()

In [ ]:
%%bash
rm -rf fes
mkdir fes
plumed sum_hills --hills hills.data --mintozero --kt 2.494353 --idw dist --outfile fes/fes.dist.data
plumed sum_hills --hills hills.data --mintozero --kt 2.494353 --idw coord --outfile fes/fes.coord.data
plumed sum_hills --hills hills.data --mintozero   --outfile fes/fes.dist-coord.data

In [ ]:
plot_colvardata("./colvar.data",5)

In [ ]:
%%bash
mkdir fes-reweight

cat <<EOF > fes-reweight/plumed_reweight.dat
# vim:ft=plumed

dist:   READ FILE=fes-reweight/colvar_reweight.data IGNORE_TIME VALUES=dist
coord:  READ FILE=fes-reweight/colvar_reweight.data IGNORE_TIME VALUES=coord
mtd:    READ FILE=fes-reweight/colvar_reweight.data IGNORE_TIME VALUES=mtd.rbias

weights: REWEIGHT_BIAS TEMP=300 ARG=mtd.rbias

HISTOGRAM ...
  ARG=dist
  GRID_MIN=0.2
  GRID_MAX=1.5
  GRID_BIN=100
  KERNEL=DISCRETE
  LOGWEIGHTS=weights
  LABEL=hg_dist
... HISTOGRAM

fes_dist: CONVERT_TO_FES GRID=hg_dist TEMP=300 MINTOZERO
DUMPGRID GRID=fes_dist FILE=fes-reweight/fes-reweight.dist.data FMT=%24.16e
EOF

In [ ]:
time_trim=500
subprocess.run("./trim-colvar-file.py --colvar-file colvar.data --output-file fes-reweight/colvar_reweight.data --time-min {0}".format(time_trim).split(),stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
subprocess.run("plumed driver --plumed fes-reweight/plumed_reweight.dat --noatoms".split(),stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)



In [ ]:
fes_reweight_ex2=np.loadtxt("../Exercise-2/fes-reweight/fes-reweight.dist.data")
fes_sumhills_ex2=np.loadtxt("../Exercise-2/fes/fes.mintozero.data")
fes_reweight_ex4=np.loadtxt("fes-reweight/fes-reweight.dist.data")
fes_sumhills_ex4=np.loadtxt("fes/fes.dist.data")

plt.plot(fes_reweight_ex2[:,0],fes_reweight_ex2[:,1]/kBT,'-',label="Biasing 1 CV - Reweighted")
plt.plot(fes_sumhills_ex2[:,0],fes_sumhills_ex2[:,1]/kBT,'-',label="Biasing 1 CV - From bias potential")
plt.plot(fes_reweight_ex4[:,0],fes_reweight_ex4[:,1]/kBT,'-',label="Biasing 2 CV - Reweighted")
plt.plot(fes_sumhills_ex4[:,0],fes_sumhills_ex4[:,1]/kBT,'-',label="Biasing 2 CV - From bias potential")
plt.xlabel("Na-Cl Distance [nm]")
plt.ylabel("Free Energy [kBT]")
plt.legend()